In [ ]:
!pip install -q pyngrok flask_ngrok # for colab
!pip install -q flask line-bot-sdk

In [ ]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    TemplateMessage, ConfirmTemplate, MessageAction,
    CarouselTemplate, CarouselColumn, URIAction, PostbackAction
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == 'confirm':
            template=ConfirmTemplate(
                text="你喜歡葬送的芙莉蓮嗎？",
                actions=[
                    MessageAction(label="是", text="我超喜歡芙莉蓮"),
                    MessageAction(label="否", text="我不喜歡芙莉蓮")
                ]
            )
            reply = TemplateMessage(
                alt_text="這是確認視窗",
                template=template
            )
        elif action == 'carousel':
            carousel_template = CarouselTemplate(
                columns=[
                    CarouselColumn(
                        thumbnail_image_url='https://upload.wikimedia.org/wikipedia/en/5/54/Secret_Untold_Melody_film_poster.png?20250131075336',
                        title='給我一首琴的時間',
                        text='不能說的秘密 韓國版',
                        actions=[
                            URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E4%B8%8D%E8%83%BD%E8%AF%B4%E7%9A%84%E7%A7%98%E5%AF%86_(%E9%9F%A9%E5%9B%BD%E7%94%B5%E5%BD%B1)'),
                            URIAction(label='預告片', uri='https://www.youtube.com/watch?v=d75UPKYpqwo'),
                            MessageAction(label='投票', text='我投給我一首琴的時間一票')
                        ]
                    ),
                    CarouselColumn(
                        thumbnail_image_url='https://media.vogue.com.tw/photos/67b43c63b1d6b566125342e0/1:1/w_1984,h_1984,c_limit/%E5%A6%B3%E6%98%AF%E6%88%91%E7%9C%BC%E4%B8%AD%E7%9A%84%E8%98%8B%E6%9E%9C_%E4%B8%BB%E6%B5%B7%E5%A0%B1_2025.3%E6%9C%88%E5%9C%A8%E5%8F%B0%E4%B8%8A%E6%98%A0.jpg',
                        title='妳是我眼中的蘋果',
                        text='那些年，我們一起追的女孩 韓國版',
                        actions=[
                            URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E5%A6%B3%E6%98%AF%E6%88%91%E7%9C%BC%E4%B8%AD%E7%9A%84%E8%98%8B%E6%9E%9C'),
                            URIAction(label='預告片', uri='https://www.youtube.com/watch?v=CYcimKJzCtw'),
                            MessageAction(label='投票', text='我投妳是我眼中的蘋果一票')
                        ]
                    )
                ]
            )
            reply = TemplateMessage(
                alt_text='這是一個輪播模板',
                template=carousel_template
            )
        else:
          reply = TextMessage(text=event.message.text)

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    reply
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()